In [ ]:
!pip install  git+https://github.com/TouiNishikawa/scCMGAN.git

In [22]:
import pandas as pd
import time
import os
from scCMGAN import scCMGAN
import warnings
warnings.simplefilter('ignore', FutureWarning)

In [ ]:
!git clone https://github.com/TouiNishikawa/scCMGAN.git

In [ ]:
%cd "./Example"

load data

In [19]:
# Path to table data
df = pd.read_csv('./Example/Example_train.csv', header=None) 

# Name of dataset
data_set = "Example"

df.iloc[0, 0] = "cellType"
df = df.T
df.columns = df.iloc[0]
df = df.drop(df.index[[0]])

cellType = ["cell_1",
            "cell_2",
            "cell_3"]

## Short usage

In [ ]:
# generate model
model = scCMGAN(persent = 40)

# finetuning model
model.fit(df, "cell_1", epochs=40)

# save model
model.save("saved_model.pkl")

# load model
model = scCMGAN.load("saved_model.pkl")

# generate data
synthetic_data = model.sample(10)

In [ ]:
synthetic_data

,cellType,gene_1,gene_3,gene_2,gene_9,gene_4,gene_5,gene_6,gene_7,gene_8,gene_10
0,cell_1,2.057228,0.00026,0.001989,0.000851,0.0,0.0,0.0,0.0,0.0,0.0
1,cell_1,0.699108,-0.000312,0.000924,0.000431,0.0,0.0,0.0,0.0,0.0,0.0
2,cell_1,2.823449,0.00048,0.001744,0.002113,0.0,0.0,0.0,0.0,0.0,0.0
3,cell_1,2.719038,-0.000364,0.001427,0.001498,0.0,0.0,0.0,0.0,0.0,0.0
4,cell_1,3.280815,0.000415,0.001052,0.000448,0.0,0.0,0.0,0.0,0.0,0.0
5,cell_1,3.241559,-0.001036,0.002391,0.002551,0.0,0.0,0.0,0.0,0.0,0.0
6,cell_1,3.992233,-0.002115,0.000952,0.001519,0.0,0.0,0.0,0.0,0.0,0.0
7,cell_1,1.602059,0.001179,-0.000034,0.000094,0.0,0.0,0.0,0.0,0.0,0.0
8,cell_1,1.01493,-0.000005,0.001294,0.002143,0.0,0.0,0.0,0.0,0.0,0.0
9,cell_1,2.466808,0.000324,0.00149,0.001065,0.0,0.0,0.0,0.0,0.0,0.0


## Training and generating

### N = 1

In [34]:
# persentage of cell marker genes
persent = 40 

# epoch: [epoch_start, epoch_end+1, epoch_term]
epoch_start = 50
epoch_end = 300
epoch_term = 50

for i in range(len(cellType)):
    # pick up
    ct =  cellType[i]
    df_celtype = df[df["cellType"]==ct]
    print("number of '" + ct + "': " + str(len(df_celtype)))
    for epoch in range(epoch_start, epoch_end+1, epoch_term ):
        if epoch == epoch_start:
            model = scCMGAN(persent)
        else:
            model = scCMGAN.load(data_set +'_scCMGAN'+str(persent)+'_' + ct + '_'+str(epoch-epoch_term)+'.pkl')
        
        # training
        print("epoch " + str(epoch)+" ")
        t1 = time.time() 
        model.fit(df, ct, 50)
        t2 = time.time()
        elapsed_time = t2-t1
        print(str(round(elapsed_time, 2)) + " seconds")
        
        model.save(data_set +'_scCMGAN'+str(persent)+'_' + ct + '_'+str(epoch)+'.pkl')

number of 'cell_1': 5
epoch 50 
2.11 seconds
epoch 100 
2.13 seconds
epoch 150 
2.09 seconds
epoch 200 
2.09 seconds
epoch 250 
3.1 seconds
epoch 300 
3.25 seconds
number of 'cell_2': 5
epoch 50 
2.83 seconds
epoch 100 
3.22 seconds
epoch 150 
2.12 seconds
epoch 200 
2.11 seconds
epoch 250 
4.83 seconds
epoch 300 
4.26 seconds
number of 'cell_3': 5
epoch 50 
2.49 seconds
epoch 100 
3.96 seconds
epoch 150 
3.35 seconds
epoch 200 
4.93 seconds
epoch 250 
3.78 seconds
epoch 300 
2.08 seconds


In [35]:
# num_data: [num_data_start, num_data_end+1, num_data_term]
num_data_start = 50
num_data_end = 500
num_data_term = 50

for i in range(len(cellType)):
    ct =  cellType[i]
    df_celtype = df[df["cellType"]==ct]

    for epoch in range(epoch_start, epoch_end+1, epoch_term):
        model = scCMGAN.load(data_set +'_scCMGAN'+str(persent)+'_' + ct + '_'+str(epoch)+'.pkl')

        for num_data in range(num_data_start, num_data_end+1, num_data_term):
                    t1 = time.time() 
                    synthetic_data = model.sample(num_data)
                    t2 = time.time()
                    elapsed_time = t2-t1
                    print(ct +"_"+ str(epoch) +"_"+str(num_data) + ":" + str(elapsed_time))
                    synthetic_data.to_csv("./synthetic_data_"+data_set+"_scCMGAN"+str(persent)+"_"+ ct +"_"+str(epoch)+"_" +  str(num_data) + ".csv", index= False)

cell_1_50_50:0.05635714530944824
cell_1_50_100:0.09379363059997559
cell_1_50_150:0.13668107986450195
cell_1_50_200:0.18702435493469238
cell_1_50_250:0.22069644927978516
cell_1_50_300:0.26413464546203613
cell_1_50_350:0.2805821895599365
cell_1_50_400:0.3387613296508789
cell_1_50_450:0.3730144500732422
cell_1_50_500:0.5162253379821777
cell_1_100_50:0.09918999671936035
cell_1_100_100:0.1510305404663086
cell_1_100_150:0.21920490264892578
cell_1_100_200:0.2628512382507324
cell_1_100_250:0.31287050247192383
cell_1_100_300:0.31076669692993164
cell_1_100_350:0.25878262519836426
cell_1_100_400:0.33481621742248535
cell_1_100_450:0.346343994140625
cell_1_100_500:0.4265165328979492
cell_1_150_50:0.08345413208007812
cell_1_150_100:0.13989973068237305
cell_1_150_150:0.1800398826599121
cell_1_150_200:0.22952961921691895
cell_1_150_250:0.2682950496673584
cell_1_150_300:0.3426089286804199
cell_1_150_350:0.711085319519043
cell_1_150_400:1.399956464767456
cell_1_150_450:1.4835436344146729
cell_1_150_500:

### N = 2 

In [36]:
# persentage of cell marker genes
persent = 20

# epoch: [epoch_start, epoch_end+1, epoch_term]
epoch_start = 50
epoch_end = 300
epoch_term = 50

for i in range(len(cellType)):
    # pick up
    ct =  cellType[i]
    df_celtype = df[df["cellType"]==ct]

    for epoch in range(epoch_start, epoch_end+1, epoch_term ):
        if epoch == epoch_start:
            model = scCMGAN(persent)
        else:
            model = scCMGAN.load(data_set +'_scCMGAN'+str(persent)+'_' + ct + '_'+str(epoch-epoch_term)+'.pkl')
        
        # training
        print("number of '" + ct + "': " + str(len(df_celtype)))
        print("epoch " + str(epoch)+" ")
        t1 = time.time() 
        model.fit(df, ct, 50)
        t2 = time.time()
        elapsed_time = t2-t1
        print(str(round(elapsed_time, 2)) + " seconds")
        
        model.save(data_set +'_scCMGAN'+str(persent)+'_' + ct + '_'+str(epoch)+'.pkl')

number of 'cell_1': 5
epoch 50 
2.01 seconds
number of 'cell_1': 5
epoch 100 
2.3 seconds
number of 'cell_1': 5
epoch 150 
2.96 seconds
number of 'cell_1': 5
epoch 200 
3.08 seconds
number of 'cell_1': 5
epoch 250 
1.96 seconds
number of 'cell_1': 5
epoch 300 
1.92 seconds
number of 'cell_2': 5
epoch 50 
1.94 seconds
number of 'cell_2': 5
epoch 100 
1.94 seconds
number of 'cell_2': 5
epoch 150 
1.94 seconds
number of 'cell_2': 5
epoch 200 
2.97 seconds
number of 'cell_2': 5
epoch 250 
3.02 seconds
number of 'cell_2': 5
epoch 300 
2.34 seconds
number of 'cell_3': 5
epoch 50 
1.94 seconds
number of 'cell_3': 5
epoch 100 
1.93 seconds
number of 'cell_3': 5
epoch 150 
2.04 seconds
number of 'cell_3': 5
epoch 200 
1.97 seconds
number of 'cell_3': 5
epoch 250 
2.54 seconds
number of 'cell_3': 5
epoch 300 
2.98 seconds


In [37]:
# num_data: [num_data_start, num_data_end+1, num_data_term]
num_data_start = 50
num_data_end = 500
num_data_term = 50

for i in range(len(cellType)):
    ct =  cellType[i]
    df_celtype = df[df["cellType"]==ct]

    for epoch in range(epoch_start, epoch_end+1, epoch_term):
        model = scCMGAN.load(data_set +'_scCMGAN'+str(persent)+'_' + ct + '_'+str(epoch)+'.pkl')

        for num_data in range(num_data_start, num_data_end+1, num_data_term):
                    t1 = time.time() 
                    synthetic_data = model.sample(num_data)
                    t2 = time.time()
                    elapsed_time = t2-t1
                    print(ct +"_"+ str(epoch) +"_"+str(num_data) + ":" + str(elapsed_time))
                    synthetic_data.to_csv("./synthetic_data_"+data_set+"_scCMGAN"+str(persent)+"_"+ ct +"_"+str(epoch)+"_" +  str(num_data) + ".csv", index= False)

cell_1_50_50:0.07809233665466309
cell_1_50_100:0.13542699813842773
cell_1_50_150:0.19570660591125488
cell_1_50_200:0.23211264610290527
cell_1_50_250:0.29003190994262695
cell_1_50_300:0.32492947578430176
cell_1_50_350:0.3929779529571533
cell_1_50_400:0.36513519287109375
cell_1_50_450:0.3462553024291992
cell_1_50_500:0.3808445930480957
cell_1_100_50:0.05083727836608887
cell_1_100_100:0.08785343170166016
cell_1_100_150:0.13852334022521973
cell_1_100_200:0.1576378345489502
cell_1_100_250:0.19312691688537598
cell_1_100_300:0.22106289863586426
cell_1_100_350:0.2842130661010742
cell_1_100_400:0.3176088333129883
cell_1_100_450:0.3116731643676758
cell_1_100_500:0.4033515453338623
cell_1_150_50:0.0549921989440918
cell_1_150_100:0.091522216796875
cell_1_150_150:0.13340353965759277
cell_1_150_200:0.14987945556640625
cell_1_150_250:0.1864607334136963
cell_1_150_300:0.20871186256408691
cell_1_150_350:0.2934584617614746
cell_1_150_400:0.3209538459777832
cell_1_150_450:0.3315465450286865
cell_1_150_50

## Integrate synthetic_data and original data

In [38]:
# persentage of cell marker genes (N =2)
persentage_1 = 40
persentage_2 = 20

# epoch: [epoch_start, epoch_end+1, epoch_term]
epoch_start = 50
epoch_end = 300
epoch_term = 50

# num_data: [num_data_start, num_data_end+1, num_data_term]
num_data_start = 50
num_data_end = 500
num_data_term = 50

# new sample name
sample_name  ="indiv3"

In [39]:
import pandas as pd
def train2train_aug(train_data_path, pheno_data_path, num_data, epoch, method, data_set):
    df = pd.read_csv(train_data_path, header=None)
    df.iloc[0, 0] = "cellType"
    df = df.T
    df.columns = df.iloc[0]
    df = df.drop(df.index[[0]])

    for ct in cellType:
        synthetic_data_ct = pd.read_csv("./synthetic_data_"+data_set+"_scCMGAN"+"20_"+ ct +"_"+str(epoch)+"_" +  str(num_data) + ".csv")
        synthetic_data_ct_2 = pd.read_csv("./synthetic_data_"+data_set+"_scCMGAN"+"40_"+ ct +"_"+str(epoch)+"_" +  str(num_data) + ".csv")
        if ct ==  cellType[0]:
            synthetic_data = pd.concat([synthetic_data_ct, synthetic_data_ct_2])
        else:    
            synthetic_data = pd.concat([synthetic_data, synthetic_data_ct, synthetic_data_ct_2])

    df_concat_multi = pd.concat([df, synthetic_data])
    df_concat_multi_T = df_concat_multi.T
    df_concat_multi_T[1:] = df_concat_multi_T[1:].round()
    df_concat_multi_T[1:] = df_concat_multi_T[1:].astype(int)

    df = pd.read_csv(train_data_path, header=None)
    gene_pd = df[0]
    df_concat_multi_T.set_axis(gene_pd, inplace=True)

    sh=df_concat_multi_T.shape
    df_concat_multi_T.columns=range(sh[1])
    df_concat_multi_T.columns = df_concat_multi_T.iloc[0]
    df_concat_multi_T = df_concat_multi_T.drop(df_concat_multi_T.index[[0]])

    df_concat_multi_T = df_concat_multi_T.clip(lower=0)

    # save
    df_concat_multi_T.to_csv('./'+method+'/'+data_set+ '_train_'+ method + "_" +str(epoch)+ "_" + str(num_data*2) + '.csv', header=True, index=True)


    # pheno data
    df_p = pd.read_csv(pheno_data_path)
    indiv = [sample_name]
    indiv *= df_concat_multi_T.shape[1] - df.shape[1]+1
    df_p_new = pd.DataFrame(
        data={'cellID': list(range(1,num_data*2*len(cellType)+1)), 
            'cellType': synthetic_data["cellType"],
            'sampleID': indiv}
    )


    df_p_concat = pd.concat([df_p, df_p_new])
    df_p_concat[""] = list(range(1,df_p_concat.shape[0]+1))
    df_p_concat = df_p_concat.reindex(columns=['', 'cellID', 'cellType','sampleID'])

    # save
    df_p_concat.to_csv('./'+method+'/'+data_set+ '_pDataC_'+ method + "_"+ str(epoch)+ "_" + str(num_data*2) + '.csv', header=True, index=False)

In [42]:
method = "scCMGAN"
num_datas = list(range(num_data_start, num_data_end+1, num_data_term)) 
epochs = list(range(epoch_start, epoch_end+1, epoch_term))

if not os.path.exists(method):
    os.makedirs(method)

for num_data in num_datas:
    for epoch in epochs:
            print("num_data: " + str(num_data)+", epoch: " + str(epoch))
            train2train_aug('./' + data_set + '_train.csv','./' + data_set + '_pDataC_train.csv', num_data, epoch, method, data_set)

num_data: 50, epoch: 50
num_data: 50, epoch: 100
num_data: 50, epoch: 150
num_data: 50, epoch: 200
num_data: 50, epoch: 250
num_data: 50, epoch: 300
num_data: 100, epoch: 50
num_data: 100, epoch: 100
num_data: 100, epoch: 150
num_data: 100, epoch: 200
num_data: 100, epoch: 250
num_data: 100, epoch: 300
num_data: 150, epoch: 50
num_data: 150, epoch: 100
num_data: 150, epoch: 150
num_data: 150, epoch: 200
num_data: 150, epoch: 250
num_data: 150, epoch: 300
num_data: 200, epoch: 50
num_data: 200, epoch: 100
num_data: 200, epoch: 150
num_data: 200, epoch: 200
num_data: 200, epoch: 250
num_data: 200, epoch: 300
num_data: 250, epoch: 50
num_data: 250, epoch: 100
num_data: 250, epoch: 150
num_data: 250, epoch: 200
num_data: 250, epoch: 250
num_data: 250, epoch: 300
num_data: 300, epoch: 50
num_data: 300, epoch: 100
num_data: 300, epoch: 150
num_data: 300, epoch: 200
num_data: 300, epoch: 250
num_data: 300, epoch: 300
num_data: 350, epoch: 50
num_data: 350, epoch: 100
num_data: 350, epoch: 150